The following is a tutorial on using the new IOPro PostgreSQL/PostGIS/Greenplum adapter. In order to run this notebook, you'll need a running PostgreSQL server with PostGIS extensions installed. Otherwise you can simply follow along with the saved example results. All the examples here were tested with Python 3.5, but they should also work with Python 2.7

First we need to generate some test data. Since IOPro only supports pulling data from data sources, we'll use the Python module psycopg2 to create and populate a new table. The connection parameters will need to be changed to run this for your own PostgreSQL server (preferably a non production server!). The following example will create a new database called 'iopro_tutorial'. If a database already exists with that name, it will be destroyed.

In [51]:
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

conn = psycopg2.connect('host=localhost dbname=postgres user=jayvius')

# A new database cannot be created inside a transaction block,
# so enable autocommit
conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)

# Use a try/except block to guard against existing database
# with same name. If database already exists, drop it and
# create new one.
cursor = conn.cursor()
try:
    cursor.execute('CREATE DATABASE iopro_tutorial')
except psycopg2.ProgrammingError:
    cursor.execute('DROP DATABASE iopro_tutorial')
    cursor.execute('CREATE DATABASE iopro_tutorial')
cursor.close()
conn.close()

conn = psycopg2.connect('host=localhost dbname=iopro_tutorial user=jayvius')
        
cursor = conn.cursor()

# Enable PostGIS extensions for this database
cursor.execute('CREATE EXTENSION postgis')
cursor.execute('CREATE EXTENSION postgis_topology')
cursor.execute('CREATE EXTENSION fuzzystrmatch')
cursor.execute('CREATE EXTENSION postgis_tiger_geocoder')

cursor.execute('CREATE TABLE data (integer integer, '
                                  'real real, '
                                  'string varchar(10), '
                                  'point2d geometry(POINT), '
                                  'multipoint3d geometry(MULTIPOINTZ))')
cursor.execute("INSERT INTO data (integer, real, string, point2d, multipoint3d) VALUES "
               "(1, 1.1, 'a', ST_MakePoint(0, 1), ST_Collect(ARRAY[ST_MakePoint(0, 1, 2), ST_MakePoint(3, 4, 5)]))")
cursor.execute("INSERT INTO data (integer, real, string, point2d, multipoint3d) VALUES "
               "(2, 2.2, 'bbb', ST_MakePoint(2, 3), ST_Collect(ARRAY[ST_MakePoint(6, 7, 8), ST_MakePoint(9, 10, 11)]))")
cursor.execute("INSERT INTO data (integer, real, string, point2d, multipoint3d) VALUES "
               "(3, 3.3, 'cccccc', ST_MakePoint(4, 5), ST_Collect(ARRAY[ST_MakePoint(12, 13, 14), ST_MakePoint(15, 16, 17)]))")
cursor.execute("INSERT INTO data (integer, real, string, point2d, multipoint3d) VALUES "
               "(4, 4.4, 'ddddddddd', ST_MakePoint(6, 7), ST_Collect(ARRAY[ST_MakePoint(18, 19, 20), ST_MakePoint(21, 22, 23)]))")
cursor.execute("INSERT INTO data (integer, real, string, point2d, multipoint3d) VALUES "
               "(5, 5.5, 'eeeeeeeeee', ST_MakePoint(8, 9), ST_Collect(ARRAY[ST_MakePoint(24, 25, 26), ST_MakePoint(27, 28, 29)]))")

conn.commit()
cursor.close()
conn.close()

Now we can start using IOPro. First we'll create a new PostgreSQL adapter for the table we created above.

In [2]:
import iopro
adapter = iopro.PostgresAdapter('host=localhost dbname=iopro_tutorial user=jayvius', table='data')

IOPro adapters use slicing to retrieve data. To retrieve all records from the 'data' table, the standard NumPy slicing notation can be used:

In [2]:
adapter[:]

array([ (1, 1.100000023841858, 'a', [0.0, 1.0], [(0.0, 1.0, 2.0), (3.0, 4.0, 5.0)]),
       (2, 2.200000047683716, 'bbb', [2.0, 3.0], [(6.0, 7.0, 8.0), (9.0, 10.0, 11.0)]),
       (3, 3.299999952316284, 'cccccc', [4.0, 5.0], [(12.0, 13.0, 14.0), (15.0, 16.0, 17.0)]),
       (4, 4.400000095367432, 'ddddddddd', [6.0, 7.0], [(18.0, 19.0, 20.0), (21.0, 22.0, 23.0)]),
       (5, 5.5, 'eeeeeeeeee', [8.0, 9.0], [(24.0, 25.0, 26.0), (27.0, 28.0, 29.0)])], 
      dtype=[('integer', '<i4'), ('real', '<f4'), ('string', '<U10'), ('point2d', '<f8', (2,)), ('multipoint3d', 'O')])

Notice that the point2d GIS type is stored as two real values. Also notice that the multipoint3d GIS type is stored as a list of tuples of three real values, since the number of 3d points can vary with each record.

The PostgreSQL adapter has a few properties that we can use to find out more about our data. We can get the number of records in our dataset:

In [3]:
adapter.num_records

5

or the number of fields:

In [4]:
adapter.num_fields

5

We can also find the names of each field:

In [5]:
adapter.field_names

['integer', 'real', 'string', 'point2d', 'multipoint3d']

These names come from the names of the columns in the database and are used by default for the field names in our NumPy array result. We can change these names by setting the field_names property using a list of field names:

In [6]:
adapter.field_names = ['field1', 'field2', 'field3', 'field4', 'field5']
adapter[:].dtype

dtype([('field1', '<i4'), ('field2', '<f4'), ('field3', '<U10'), ('field4', '<f8', (2,)), ('field5', 'O')])

We can also set individual fields by using a dict, where the key is the field number and the value is the field name we want:

In [7]:
adapter.field_names = {1: 'AAA'}
adapter[:].dtype

dtype([('integer', '<i4'), ('AAA', '<f4'), ('string', '<U10'), ('point2d', '<f8', (2,)), ('multipoint3d', 'O')])

We can also find out the NumPy dtype of each field:

In [8]:
adapter.field_types

['i4', 'f4', 'U10', 'f8', 'O']

Similar to the field_names property, we can set this property using a list or dict to force a field to be cast to a specific type:

In [9]:
adapter.field_types = {0: 'f4', 1: 'i4', 2: 'U3', 4: 'O'}
adapter[:]

array([(1.0, 1, 'a', [0.0, 1.0], [(0.0, 1.0, 2.0), (3.0, 4.0, 5.0)]),
       (2.0, 2, 'bbb', [2.0, 3.0], [(6.0, 7.0, 8.0), (9.0, 10.0, 11.0)]),
       (3.0, 3, 'ccc', [4.0, 5.0], [(12.0, 13.0, 14.0), (15.0, 16.0, 17.0)]),
       (4.0, 4, 'ddd', [6.0, 7.0], [(18.0, 19.0, 20.0), (21.0, 22.0, 23.0)]),
       (5.0, 5, 'eee', [8.0, 9.0], [(24.0, 25.0, 26.0), (27.0, 28.0, 29.0)])], 
      dtype=[('integer', '<f4'), ('AAA', '<i4'), ('string', '<U3'), ('point2d', '<f8', (2,)), ('multipoint3d', 'O')])

Notice that the integer field has been cast to a float field, the float field has been cast to an integer field, and the string field has been cast from 10 characters to 3 characters resulting in truncation.

IOPro adapters support all the basic slicing features. For example, to retrieve the first three records:

In [11]:
adapter[0:3]

array([(1.0, 1, 'a', [0.0, 1.0], [(0.0, 1.0, 2.0), (3.0, 4.0, 5.0)]),
       (2.0, 2, 'bbb', [2.0, 3.0], [(6.0, 7.0, 8.0), (9.0, 10.0, 11.0)]),
       (3.0, 3, 'ccc', [4.0, 5.0], [(12.0, 13.0, 14.0), (15.0, 16.0, 17.0)])], 
      dtype=[('integer', '<f4'), ('AAA', '<i4'), ('string', '<U3'), ('point2d', '<f8', (2,)), ('multipoint3d', 'O')])

retrieve the last record:

In [12]:
adapter[-1]

array([(5.0, 5, 'eee', [8.0, 9.0], [(24.0, 25.0, 26.0), (27.0, 28.0, 29.0)])], 
      dtype=[('integer', '<f4'), ('AAA', '<i4'), ('string', '<U3'), ('point2d', '<f8', (2,)), ('multipoint3d', 'O')])

retrieve every other record from the first four records:

In [13]:
adapter[0:4:2]

array([(1.0, 1, 'a', [0.0, 1.0], [(0.0, 1.0, 2.0), (3.0, 4.0, 5.0)]),
       (3.0, 3, 'ccc', [4.0, 5.0], [(12.0, 13.0, 14.0), (15.0, 16.0, 17.0)])], 
      dtype=[('integer', '<f4'), ('AAA', '<i4'), ('string', '<U3'), ('point2d', '<f8', (2,)), ('multipoint3d', 'O')])

We can also filter the fields that we want to retrieve by passing a list of field names to the constructor:

In [14]:
adapter = iopro.PostgresAdapter('host=localhost dbname=iopro_tutorial user=jayvius',
                                table='data',
                                field_filter=['integer', 'string'])
adapter[:]

array([(1, 'a'), (2, 'bbb'), (3, 'cccccc'), (4, 'ddddddddd'),
       (5, 'eeeeeeeeee')], 
      dtype=[('integer', '<i4'), ('string', '<U10')])

For fields like multipoint3d with a variable length, the adapter will return values as a list of tuples containing the float components of each point. For improved performance, a field shape can be specified which will set the max dimensions of the field values. For example, for our multipoint3d field we can it to have a maximum of two points so that each set of 3d points will be stored in a 2x3 subarray of floats:

In [13]:
adapter = iopro.PostgresAdapter('host=localhost dbname=iopro_tutorial user=jayvius',
                                table='data',
                                field_filter=['multipoint3d'],
                                field_shapes={'multipoint3d': 2})
adapter[:]

array([([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0]],),
       ([[6.0, 7.0, 8.0], [9.0, 10.0, 11.0]],),
       ([[12.0, 13.0, 14.0], [15.0, 16.0, 17.0]],),
       ([[18.0, 19.0, 20.0], [21.0, 22.0, 23.0]],),
       ([[24.0, 25.0, 26.0], [27.0, 28.0, 29.0]],)], 
      dtype=[('multipoint3d', '<f8', (2, 3))])

For more advanced queries, a custom select query can be passed to the constructor. Either a table name or a custom query can be passed to the constructor, but not both.

In [14]:
adapter = iopro.PostgresAdapter('host=localhost dbname=iopro_tutorial user=jayvius',
                                query='select integer, string from data where data.integer > 2')
adapter[:]

array([(3, 'cccccc'), (4, 'ddddddddd'), (5, 'eeeeeeeeee')], 
      dtype=[('integer', '<i4'), ('string', '<U10')])

Data can also be returned as a pandas dataframe using the adapter constructor's 'dataframe' argument:

In [15]:
adapter = iopro.PostgresAdapter('host=localhost dbname=iopro_tutorial user=jayvius',
                                table='data',
                                dataframe=True)
adapter[:]

,integer,real,string,point2d,multipoint3d
0,1,1.1,a,"[(0.0, 1.0)]","[(0.0, 1.0, 2.0), (3.0, 4.0, 5.0)]"
1,2,2.2,bbb,"[(2.0, 3.0)]","[(6.0, 7.0, 8.0), (9.0, 10.0, 11.0)]"
2,3,3.3,cccccc,"[(4.0, 5.0)]","[(12.0, 13.0, 14.0), (15.0, 16.0, 17.0)]"
3,4,4.4,ddddddddd,"[(6.0, 7.0)]","[(18.0, 19.0, 20.0), (21.0, 22.0, 23.0)]"
4,5,5.5,eeeeeeeeee,"[(8.0, 9.0)]","[(24.0, 25.0, 26.0), (27.0, 28.0, 29.0)]"


For one last example, we're use the adapter constructor's query argument to retrieve some PostGIS data that falls within a given bounding box:

In [16]:
adapter = iopro.PostgresAdapter('host=localhost dbname=iopro_tutorial user=jayvius',
                                query='select integer, point2d from data '
                                      'where data.point2d @ ST_MakeEnvelope(0, 0, 4, 4)')
adapter[:]

array([(1, [0.0, 1.0]), (2, [2.0, 3.0])], 
      dtype=[('integer', '<i4'), ('point2d', '<f8', (2,))])

In [17]:
# Force database connection to close
del adapter